# Stock Price Predictor with Long Short Term Memory (LSTM)

Hello hello! I'm Nathan a Year 3 Chemical Engineer with Second Major in Data Analytics! This is one of my projects I did for fun which allows you to:
- Predict the **next-day closing price** for any stock
- Get a suggested action: **BUY / SELL / HOLD**
- Visualize the predictions made
- This can also be applied to any stock (just need to find their respective ticker

## 1. Import Libraries

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

## 2. Choose Your Stock of Interest

Type in the **stock ticker symbol** you want to analyze (e.g., `AAPL`, `TSLA`, `MSFT`, `GOOG`) and continue to the next part

**VOO** is set as the default so the notebook still works even if you don’t enter anything.

In [4]:
ticker_input = widgets.Text(value='VOO', description='Ticker:', placeholder='Enter stock symbol (e.g. AAPL)', 
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

display(ticker_input)

# Read and format user input
ticker = ticker_input.value.upper()

Text(value='VOO', description='Ticker:', layout=Layout(width='300px'), placeholder='Enter stock symbol (e.g. A…

## 3. Downloading Historical Data (Source: Yahoo Finance)

In [25]:
start_date = '2024-01-01'
end_date = datetime.today() - timedelta(days=1)
ticker = ticker_input.value.upper()
data = yf.download(ticker, start=start_date, end=end_date.strftime('%Y-%m-%d'))
data = data[['Close']].dropna()

[*********************100%***********************]  1 of 1 completed


In [26]:
#Use this to check whether data has been imported #
data.head()

Price,Close
Ticker,V
Date,
2024-01-02,256.049530
2024-01-03,255.169205
2024-01-04,256.781433
2024-01-05,256.860596
2024-01-08,259.679535


## 4. Feature Engineering


In [27]:
#Since we are using stock prices data, scaling it allows for easier training of the model #
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
lookback = 60
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    y.append(scaled_data[i])

X = np.array(X)
y = np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

## 5. Model Building and Training 

In [28]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)), #50 units with return sequences since we are stacking#
    Dropout(0.2), #Dropouts set to 0.2 to prevent any overfitting#
    LSTM(50),
    Dropout(0.2),
    Dense(1) #Final layer which has only 1 output which is the price we want to find out about#
])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X, y, epochs=20, batch_size=32, verbose=0)

C:\Users\65859\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,955 (359.20 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 61,304 (239.47 KB)

## 6. Model Evaluation


In [30]:
#Using an 80-20 train test split#
split = int(len(scaled_data) * 0.8)
train_data = scaled_data[:split]
test_data = scaled_data[split - lookback:] 
X_test, y_test = [], []
for i in range(lookback, len(test_data)):
    X_test.append(test_data[i-lookback:i])
    y_test.append(test_data[i])
X_test = np.array(X_test).reshape(-1, lookback, 1)
y_test = np.array(y_test)
predicted_scaled = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_scaled)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/stepWARNING:tensorflow:6 out of the last 10 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001A3CD529E40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


In [31]:
rmse = np.sqrt(mean_squared_error(real_prices, predicted_prices))
r2 = r2_score(real_prices, predicted_prices)
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

RMSE: 10.90
R² Score: 0.1950


## 7. Predicting Tomorrow's Stock Closing Price


In [32]:
#Using the last 60 days as reference, we can try to use the model to predict tomorrow's stock closing price#
last_60 = scaled_data[-lookback:]
last_60 = np.reshape(last_60, (1, lookback, 1))
next_scaled = model.predict(last_60)[0][0]
next_price = scaler.inverse_transform([[next_scaled]])[0][0]

#I tried to make a simple indication on what to do based on change but of course we must also do our research#
today_price = float(data['Close'].iloc[-1])
change = (next_price - today_price) / today_price

if change > 0.005:
    signal = 'BUY'
elif change < -0.005:
    signal = 'SELL'
else:
    signal = 'HOLD'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [33]:
#Finally, here's a simple summary for you #
print(f"Today's Close: ${today_price:.2f}")
print(f"Predicted Close (Next Day): ${next_price:.2f}")
print(f"Suggested Action: {signal}")

Today's Close: $367.90
Predicted Close (Next Day): $356.54
Suggested Action: SELL
